# How to Train a YOLOv8 Oriented Bounding Boxes Model

Reference: [YOLOv8 oriented bounding box training blog post](https://blog.roboflow.com/train-yolov8-obb-model/).

In [1]:
# Install packages

!pip install Pillow==9.0.0 ultralytics roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
INFO: pip is looking at multiple versions of pi-heif to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of pi-heif to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 13.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 33.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 38.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
 

In [2]:
# Check YOLO package can be loaded properly

import ultralytics
ultralytics.checks()

Ultralytics 8.3.170 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla P40, 24439MiB)
Setup complete ✅ (8 CPUs, 70.6 GB RAM, 126.6/292.1 GB disk)


In [4]:
# Download dataset from Roboflow

!mkdir {HOME}/datasets
%cd {HOME}/datasets

import roboflow

roboflow.login()

rf = roboflow.Roboflow()

project = rf.workspace("ftc21229").project("perspective-transformed-samples")
dataset = project.version(9).download("yolov8-obb")

import yaml

with open(f'{dataset.location}/data.yaml', 'r') as file:
    data = yaml.safe_load(file)

data['path'] = dataset.location

with open(f'{dataset.location}/data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

mkdir: cannot create directory ‘{HOME}/datasets’: No such file or directory
[Errno 2] No such file or directory: '{HOME}/datasets'
/content
visit https://app.roboflow.com/auth-cli to get your authentication token.
Paste the authentication token here: ········
loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov8-obb in progress : 85.0%
Version export complete for yolov8-obb format



Extracting Dataset Version Zip to Perspective-Transformed-Samples-9 in yolov8-obb:: 100%|██████████| 304/304 [00:00<00:00, 5354.84it/s]


In [5]:
# Preprocess dataset

import cv2
import os

model_image_size = (320, 480)

def merge_label(line):
    fields = line.split(' ')
    fields[0] = '0'
    return ' '.join(fields)

def merge_labels(labels_dir):
    files = [x for x in os.listdir(labels_dir) if x.endswith('.txt')]
    for file in files:
        path = f'{labels_dir}/{file}'
        with open(path) as f:
            lines = f.readlines()
        lines = [merge_label(x) for x in lines]
        with open(path, 'w') as f:
            f.writelines(lines)
    print(f'merged labels {len(files)} files under {labels_dir}')

def resize_images(images_dir):
    files = [x for x in os.listdir(images_dir) if x.endswith('.jpg')]
    for file in files:
        path = f'{images_dir}/{file}'
        img = cv2.imread(path)
        img = cv2.resize(img, model_image_size, interpolation=cv2.INTER_AREA)
        cv2.imwrite(path, img)
    print(f'resized images {len(files)} files under {images_dir}')

def preprocess():
    for dir in ['train', 'valid', 'test']:
        merge_labels(f'{dataset.location}/{dir}/labels')
        cache_file = f'{dataset.location}/{dir}/labels.cache'
        if os.path.exists(cache_file):
            os.remove(cache_file)
        resize_images(f'{dataset.location}/{dir}/images')

preprocess()

merged labels 126 files under /content/Perspective-Transformed-Samples-9/train/labels
resized images 126 files under /content/Perspective-Transformed-Samples-9/train/images
merged labels 13 files under /content/Perspective-Transformed-Samples-9/valid/labels
resized images 13 files under /content/Perspective-Transformed-Samples-9/valid/images
merged labels 7 files under /content/Perspective-Transformed-Samples-9/test/labels
resized images 7 files under /content/Perspective-Transformed-Samples-9/test/images


In [6]:
# Train OBB model

from ultralytics import YOLO

model = YOLO('yolov8n-obb.pt')

# Image size is internally padded to 160x256 for training
results = model.train(data=f"{dataset.location}/data.yaml", epochs=100, imgsz=(320, 480))

100%|██████████| 6.26M/6.26M [00:00<00:00, 36.6MB/s]


Ultralytics 8.3.170 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla P40, 24439MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Perspective-Transformed-Samples-9/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=(320, 480), int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-obb.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0

100%|██████████| 755k/755k [00:00<00:00, 20.0MB/s]

Overriding model.yaml nc=15 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


...

    100/100      1.55G      0.481     0.3405      1.054        223        480: 100%|██████████| 8/8 [00:01<00:00,  6.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.48it/s]

                   all         13        256      0.987      0.973      0.994      0.905



100 epochs completed in 0.055 hours.
Optimizer stripped from runs/obb/train/weights/last.pt, 6.5MB
Optimizer stripped from runs/obb/train/weights/best.pt, 6.5MB

Validating runs/obb/train/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla P40, 24439MiB)
YOLOv8n-obb summary (fused): 81 layers, 3,077,804 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.36it/s]
Exception in thread Thread-37 (plot_images):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/utils/plotting.py", line 788, in plot_images
    annotator.box_label(box, label, color=color)
  File "/usr/local/lib/python3.10/dist-packages/ultralytics/utils/plotting.py", line 325, in box_label
    self.draw.polygon(
  File "/usr/local/lib/python3.10/dist-packages/PIL/ImageDraw.py", line 257, in polygon
Exception in thread Thread-38 (plot_images):
Traceback (most recent call last):
  File "/usr

                   all         13        256      0.988      0.979      0.994      0.908
                  blue         13        256      0.988      0.979      0.994      0.908
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs/obb/train


In [12]:
# Test model with post-processing

import cv2
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from ultralytics import YOLO

image_size = (640, 360)
warped_image_size = (300, 450)

image_points = np.array([
    [173.9,  48.7],
    [393.5,  47.7],
    [450.2, 258.4],
    [ 98.4, 250.4]
], dtype=np.float32)

object_points = np.array([
    [50, 150],
    [250, 150],
    [250, 350],
    [50, 350]
], dtype=np.float32)

camera_matrix = np.array([
    [345.96589147568955, 0.0, 311.7387017952808],
    [0.0, 347.85001143507657, 173.23578163143304],
    [0.0, 0.0, 1.0]
], dtype=np.float32)

dist_coeffs = np.array([
    -0.30485567438668043,
    0.07975532745501325,
    -0.0003135652017283016,
    -0.00207616958375859,
    0.011229097994240867
], dtype=np.float32)

def display_image(*images):
    for image in images:
        height, width = image.shape[:2]
        image_widget = widgets.Image(format='jpg', width=width, height=height)
        image_widget.value = bytes(cv2.imencode('.jpg', image)[1])
        display(image_widget)

def perspective(image):
    undistorted_points = cv2.undistortPoints(image_points,
        camera_matrix, dist_coeffs, None, camera_matrix).reshape(-1, 2)
    M = cv2.getPerspectiveTransform(undistorted_points, object_points)
    image = cv2.resize(image, image_size, interpolation=cv2.INTER_AREA)
    undistorted = cv2.undistort(image, camera_matrix, dist_coeffs)
    warped = cv2.warpPerspective(undistorted, M, warped_image_size)
    cv2.polylines(image, [image_points.astype(np.int32).reshape(-1, 1, 2)],
        isClosed=True, color=(0, 255, 0), thickness=1)
    cv2.polylines(undistorted, [undistorted_points.astype(np.int32).reshape(-1, 1, 2)],
        isClosed=True, color=(0, 255, 0), thickness=1)
    return image, undistorted, warped

def color_segmentation(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([80, 100, 10])
    upper_blue = np.array([140, 255, 255])
    lower_red1 = np.array([0, 30, 100])
    upper_red1 = np.array([15, 255, 255])
    lower_red2 = np.array([160, 30, 100])
    upper_red2 = np.array([180, 255, 255])
    mask = np.zeros_like(image[:, :, 0])
    mask += cv2.inRange(hsv, lower_blue, upper_blue)
    mask += cv2.inRange(hsv, lower_red1, upper_red1)
    mask += cv2.inRange(hsv, lower_red2, upper_red2)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=3)
    segmented = cv2.bitwise_and(image, image, mask=mask)
    return mask, segmented

def masked_ratio(box, mask):
    box_mask = np.zeros_like(mask)
    cv2.fillPoly(box_mask, [box.reshape((-1, 1, 2))], 255)
    intersection = cv2.bitwise_and(mask, box_mask)
    box_area = np.count_nonzero(box_mask)
    intersection_area = np.count_nonzero(intersection)
    return intersection_area / box_area

def wh_ratio(xywhr):
    _, _, w, h, _ = xywhr
    ratio = w / h
    if ratio < 1:
        ratio = 1 / ratio
    return ratio

def yolo(model_file, image, mask):
    model = YOLO(model_file)
    input_image_size = (image.shape[1], image.shape[0])
    model_image_size = (320, 480)
    image = cv2.resize(image, model_image_size, interpolation=cv2.INTER_AREA)
    mask = cv2.resize(mask, model_image_size, interpolation=cv2.INTER_AREA)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    detected = image.copy()
    result = model(rgb)[0]
    n = len(result.obb.xyxyxyxy)
    for i in range(n):
        conf = round(result.obb.conf[i].item() * 100) / 100
        if conf < 0.2:
            continue
        box = np.array(result.obb.xyxyxyxy[i].tolist(), dtype=np.int32)
        if masked_ratio(box, mask) < 0.8:
            continue
        if not (2.0 < wh_ratio(result.obb.xywhr[i].tolist()) < 2.6):
            continue
        cv2.polylines(detected, [box.reshape(-1, 1, 2)],
            isClosed=True, color=(0, 255, 0), thickness=1)
        cv2.putText(detected, str(conf), box[0], cv2.FONT_HERSHEY_SIMPLEX, 
            fontScale=0.3, color=(0, 255, 127), thickness=1)
    detected = cv2.resize(detected, input_image_size, interpolation=cv2.INTER_AREA)
    return detected

In [22]:
# Read from webcam

image = read_webcam()
image, undistorted, warped = perspective(image)
mask, segmented = color_segmentation(warped)
detected = yolo('runs/obb/train/weights/best.pt', warped, mask)
display_image(image, warped, segmented, detected)


0: 256x192 14.2ms
Speed: 2.3ms preprocess, 14.2ms inference, 5.1ms postprocess per image at shape (1, 3, 256, 192)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [13]:
# Read from warped image files

import os

files = [x for x in os.listdir('.') if x.endswith('.webp') or x.endswith('.jpg') or x.endswith('.jpeg')]
for f in files:
    warped = cv2.imread(f)
    mask, segmented = color_segmentation(warped)
    detected = yolo('runs/obb/train/weights/best.pt', warped, mask)
    display_image(warped, segmented, detected)


0: 480x320 None21.4ms
Speed: 3.5ms preprocess, 21.4ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 320)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…


0: 480x320 None14.9ms
Speed: 0.8ms preprocess, 14.9ms inference, 6.3ms postprocess per image at shape (1, 3, 480, 320)


...

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…


0: 480x320 None12.8ms
Speed: 0.8ms preprocess, 12.8ms inference, 4.7ms postprocess per image at shape (1, 3, 480, 320)


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…